In [ ]:
from pylab import *

import skimage
from skimage import color, io
import numpy as np
import scipy
from numba import njit

In [ ]:
imgRGB = io.imread('sekvence/street.tif')
imgRGB = imgRGB/np.max(imgRGB)

input_title = 'ulazna slika'
form = ' u RGB formatu'
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(imgRGB); axes.set_axis_off(); axes.set_title(input_title + form)

In [ ]:
imgRGB.shape[2]

Slika je prikazana u RGB formatu, možemo primetiti da su izraženi tamni pikseli, pa ćemo razmatrati sledeće opcije:

1) Primena nelinearnog preslikavanja koje će da preslika tamne piksele u širi opseg, a svetle u manji. Razmatramo preslikavanje log(1 + ks)

2) Primena tehnike ekvalizacije histograma sa i bez odsecanja

3) Primena fazi logike radi popravljanja kontrasta

4) Adaptivna evalizacija sa odsecanjem

Međutim prvi je korak da prebacimo sliku YUV format, a zatim vršimo sve transformacije nad Y slojem. 

In [ ]:
imgHSV = color.rgb2hsv(imgRGB)

fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow(imgRGB); ax[0].set_title('RGB', fontsize=14); ax[0].set_axis_off();
ax[1].imshow(imgHSV[:,:,2], vmin=0, vmax=1, cmap='gray'); ax[1].set_title('V', fontsize=14); ax[1].set_axis_off();

imgV = imgHSV[:,:,2];

In [ ]:
def intensity_transform(s, var, trans_type, plot = True, value = 256):
    
    if(trans_type == 'log'):
        k = var[0]
        t = log(1 + 10**k * s)/log(1 + 10**k);
    if(trans_type == 'linear'):
        a = var[0]; b = var[1]; val_a = var[2]; val_b = var[3]; end = var[4];

        x_val = [0, a, b, end]
        y_val = [0, val_a, val_b, end]

        t = interp(double(s), x_val, y_val);
    
    if(plot):
        plot_hist_img(s, t, value);

    return t      
        
def plot_hist_img(s, t, value):
        hist_s, bin_edges_s = np.histogram(s.flatten()*(value - 1), bins=value, range=(0,(value - 1)))
        hist_t, bin_edges_t = np.histogram(t.flatten()*(value - 1), bins=value, range=(0,(value - 1)))
        
        fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
        tight_layout();
        ax[0].imshow(s , vmin=0, vmax=1, cmap='gray'); ax[0].set_title('Ulazna slika', fontsize=14); ax[0].set_axis_off();
        ax[1].imshow(t, vmin=0, vmax=1, cmap='gray'); ax[ 1].set_title('Izlazna slika', fontsize=14); ax[1].set_axis_off();
        
        fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=120);
        plt.bar(bin_edges_s[0:-1], hist_s, color='b', label = 'ulaz')
        plt.bar(bin_edges_t[0:-1], hist_t, color='r', label = 'izlaz')
        plt.legend(); plt.title('Histogram');
        return fig, ax


In [ ]:
intensity_transform(imgV, [1], 'log');

Pola dinamičkog opsega i dalje nije iskorišćeno. Povećavamo k za 1.

In [ ]:
intensity_transform(imgV, [2], 'log');

Dolazi do prevelikog razvlačenja tamnih regiona, to kao rezultat ima narušavanje kvaliteta slike. Kao kompromis usvajamo k = 1.7

In [ ]:
k = 1.8
t1 = intensity_transform(imgV, [k], 'log');

In [ ]:
temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], t1))
img_out1 = color.hsv2rgb(temp)

fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow((imgRGB*255).astype(uint8)); ax[0].set_title('RGB ulaz', fontsize=14); ax[0].set_axis_off();
ax[1].imshow((img_out1*255).astype(uint8)); ax[1].set_title('RGB izlaz', fontsize=14); ax[1].set_axis_off();

Mozemo i da primenimo istu transf na RGB slojeve, koristimo iste parametre kao za V.

In [ ]:
R = imgRGB[:, :, 0];
G = imgRGB[:, : , 1];
B = imgRGB[:, :, 2];




r1 = intensity_transform(R, [k], 'log', False);


g1 = intensity_transform(G, [k], 'log', False);

b1 = intensity_transform(B, [k], 'log', False);


img_out2= np.dstack((r1, g1, b1));

fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow((imgRGB*255).astype(uint8)); ax[0].set_title('RGB ulaz', fontsize=14); ax[0].set_axis_off();
ax[1].imshow((img_out2*255).astype(uint8)); ax[1].set_title('RGB izlaz', fontsize=14); ax[1].set_axis_off();

Kao što je i očekivano došlo je do određene promene u bojama, zato što u RGB sloju intezitet je nelinearna fja koordinata, a boja je definisana relativnim udelom R, G i B komponente. Kao rezultat popravkom kontrasta (pogotovo primenom nelineranih funkcija) možemo narušiti relativni odnos R, G i B komponenti.

Zbog toga vršimo obradu u HSV sistemu, tu su jasno izdvojene informacije o intezitetu svetla i boji.

In [ ]:

def equalize_hist(s, clip_limit = 1, value = 256, plot = True):
    
    value = int(value)
    hist_s, bin_edges_s = np.histogram(s.flatten()*(value - 1), bins=value, range=(0,(value - 1)))
    N = sum(hist_s)
    
    scale = max(hist_s)
    
        
    if(clip_limit < 1 and clip_limit > 0):
        hist_s = clip_hist(hist_s, clip_limit, N)
    N = sum(hist_s)   
    cdf  = np.array([sum(hist_s[:i+1])  for i in range(value)])/N;
    
    x_val = np.linspace(0, 1, value);
                     
    t =  interp(double(s), x_val, cdf);
    
        
    
    if(plot):
        fig, ax = plot_hist_img(s, t, value);
        ax.plot(np.linspace(0, value-1,value), cdf*scale, 'g--')
    
    return t
        
def clip_hist(hist, clip_limit, N):
    
    ind = np.ones(len(hist)).astype(int)
    done = 0;
    max_quant = round(clip_limit*N)
    extra = 0
    itera = 0
    while not done and itera < 10:
        itera =itera + 1
        for idx, i in enumerate(hist):
            if(i >= max_quant):
                extra = extra + i - max_quant
                hist[idx]= max_quant
                ind[idx] = 0
            
        k = round(extra/sum(ind));
        hist[ind] = hist[ind] + k;

        if(max(hist) <= max_quant):
            done = 1;
    return hist


In [ ]:
t1 =  equalize_hist(imgV)

Kao što je i očekivano usled velike koncentracije crnih piksela došlo je do "izbeljenja" tamnih piksela. Primenićemo clip_limit.

In [ ]:
clip_limit = 0.02
t1 =  equalize_hist(imgV, clip_limit)

In [ ]:
temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], t1))
img_out3 = color.hsv2rgb(temp)



fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow((imgRGB*255).astype(uint8)); ax[0].set_title('RGB ulaz', fontsize=14); ax[0].set_axis_off();
ax[1].imshow((img_out3*255).astype(uint8)); ax[1].set_title('RGB izlaz', fontsize=14); ax[1].set_axis_off();

In [ ]:

r1 = equalize_hist(R, clip_limit, 256, False);
g1 = equalize_hist(G, clip_limit, 256, False);
b1 = equalize_hist(B, clip_limit, 256, False);

img_out4 = np.dstack((r1, g1, b1));

fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow((imgRGB*255).astype(uint8)); ax[0].set_title('RGB ulaz', fontsize=14); ax[0].set_axis_off();
ax[1].imshow((img_out4*255).astype(uint8)); ax[1].set_title('RGB izlaz', fontsize=14); ax[1].set_axis_off();

In [ ]:

def dark(s, val):
    x_val = val[0]
    y_val = val[1]
    return (interp(double(s), x_val, y_val))

def light(s, val):
    x_val = val[0]
    y_val = val[1]
 

    return (interp(double(s), x_val, y_val))

def gray(s, val):
    x_val = val[0]
    y_val = val[1]
    return np.array(interp(double(s), x_val, y_val))
    
def fuzzy(s, dark_val, light_val, gray_val , weight,  value = 256, plot = True):
    d = weight[0]
    l = weight[2]
    g = weight[1]
    
    t = (dark(s, dark_val)*d + light(s, light_val)*l + gray(s, gray_val)*g)/(dark(s, dark_val)+ light(s, light_val) + gray(s, gray_val))
    
    if(plot):
           plot_hist_img(s, t, value);
    return t

Prvo treba da odredimo granice tamnih, svetlih i sivih oblasti, to ćemo ustanoviti tako što ćemo posmatrati crnu, sivu i belu fuzzy masku.

In [ ]:
 dark_val, light_val, gray_val =     [[0,25/255, 1], [1,  0, 0]], [[0, 15/255, 1], [0, 0, 1]], [[0, 20/255, 1], [0,1, 0]]

fig, ax = plt.subplots(1, 3, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow(dark(imgV, dark_val) , vmin=0, vmax=1, cmap='gray'); ax[0].set_title('fuzzy dark', fontsize=14); ax[0].set_axis_off();
ax[1].imshow(light(imgV, light_val), vmin=0, vmax=1, cmap='gray'); ax[ 1].set_title('fuzzy light', fontsize=14); ax[1].set_axis_off();
ax[2].imshow(gray(imgV, gray_val), vmin=0, vmax=1, cmap='gray'); ax[ 2].set_title('fuzzy gray', fontsize=14); ax[2].set_axis_off();

Podesavanjem tezina mozemo uticati na to gde ce se grupisati piskeli u histogramu.

In [ ]:
weight = [0.03, 0.5, 1]

t1 =  fuzzy(imgV, dark_val, light_val, gray_val, weight)

In [ ]:
temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], t1))
img_out5 = color.hsv2rgb(temp)


fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow(imgRGB); ax[0].set_title('RGB ulaz', fontsize=14); ax[0].set_axis_off();
ax[1].imshow(img_out5); ax[1].set_title('RGB izlaz', fontsize=14); ax[1].set_axis_off();

In [ ]:
r1 = fuzzy(R, dark_val, light_val, gray_val, weight, plot = False)
g1 = fuzzy(G, dark_val, light_val, gray_val, weight,  plot = False)
b1 = fuzzy(B, dark_val, light_val, gray_val, weight,  plot = False)

img_out6 = np.dstack((r1, g1, b1));

fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow(imgRGB); ax[0].set_title('RGB ulaz', fontsize=14); ax[0].set_axis_off();
ax[1].imshow(img_out6); ax[1].set_title('RGB izlaz', fontsize=14); ax[1].set_axis_off();

Sledi poređenje metoda: 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0,0].imshow(img_out1); ax[0, 0].set_title('log V', fontsize=14); ax[0, 0].set_axis_off();
ax[0,1].imshow(img_out2); ax[0, 1].set_title('log RGB', fontsize=14); ax[0,1].set_axis_off();

ax[1,0].imshow(img_out3); ax[1, 0].set_title('eq clip V', fontsize=14); ax[1, 0].set_axis_off();
ax[1,1].imshow(img_out4); ax[1, 1].set_title('eq clip RGB', fontsize=14); ax[1,1].set_axis_off();

ax[2,0].imshow(img_out5); ax[2, 0].set_title('fuzzy V', fontsize=14); ax[2, 0].set_axis_off();
ax[2,1].imshow(img_out6); ax[2, 1].set_title('fuzzy RGB', fontsize=14); ax[2,1].set_axis_off();

Dobijene su praktično identične slike, pa za "najbolju" metodu proglašavamo onu koju je najlakše koristiti. U ovom slučaju to je ekv. histograma, druge metode zahtevaju mnogo više finog podešavanja. Za kraj uporedićemo rezultat sa ugrađenom funkcijom za adaptivnu ekv.

In [ ]:
[height, width] = shape(imgV)

y_out = skimage.exposure.equalize_adapthist(imgV, [height/16, width/16], clip_limit=0.045, nbins=256)

temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], y_out))
img_out10 = color.hsv2rgb(temp)

fig, ax = plt.subplots(1, 2, figsize=(12,9), dpi=120);
tight_layout();
ax[0].imshow((img_out3*255).astype(uint8)); ax[0].set_title('CLHE', fontsize=14); ax[0].set_axis_off();
ax[1].imshow((img_out10*255).astype(uint8)); ax[1].set_title('CLAHE', fontsize=14); ax[1].set_axis_off();


from PIL import Image
im = Image.fromarray((img_out3*255).astype(uint8))
im.save("street_out.jpg")

## Drugi zadatak

In [ ]:
imgRGB = skimage.img_as_float(imread('sekvence/shigatse.jpg'))

input_title = 'shigatse'
form = ' u RGB formatu'
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(imgRGB); axes.set_axis_off(); axes.set_title(input_title + form)

Primenićemo metodu koja podrazumeva pronalaženje laplasijana slike, a zatim sabiranje istog na početnu sliku. To rezultuje izoštravanjem zato što laplasijan dovodi do izražaja nagle "step-like" promene u intezitetu. Isprobaćemo i varijantu gde su dijagonalni elementi različiti od nule, kako bi se bolje video izgled laplasijana primenićemo tehnike popravke kontrasta.

In [ ]:
imgHSV = color.rgb2hsv(imgRGB)

imgV = imgHSV[:,:,2];


laplacian_mask1 = np.array([ [0,  1, 0],
                            [1, -4, 1],
                            [0,  1, 0] ])


laplacian_mask2 = np.array([ [1,  1, 1],
                            [1, -8, 1],
                            [1,  1, 1] ])

img_lap1 = scipy.ndimage.correlate(imgV, laplacian_mask1)

img_lap2 = scipy.ndimage.correlate(imgV, laplacian_mask2)


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=100)
ax = axes.ravel()

ax[0].imshow(log(1+10*abs(img_lap1))/log(11), cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Laplasijan slike bez dijagonalnih el', fontsize=16)
ax[1].imshow(log(1+10*abs(img_lap2))/log(11), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Laplasijan slike sa dijagonalnim el', fontsize=16)


Varijanta sa dijagonalnim elementima bolje povećava oštrinu slike, ali sa druge strane više pojačva šum. Ako šum bude previše izražen možemo da propustimo sliku korz adaptivni NF filtar pre izoštravanja.

In [ ]:
img_sharp = imgV - 0.4*img_lap2 

img_sharp = np.clip(img_sharp, 0, 1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(img_sharp, cmap='gray', vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)

plt.tight_layout();

Nivo šuma je prihvatljiv, bitni detalji slike su izoštreni. Primenićemo i Laplasijan bez dijagonalnih elemenata.

In [ ]:
img_sharp2 = imgV - 0.8*img_lap1

img_sharp2 = np.clip(img_sharp2, 0, 1)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(img_sharp2, cmap='gray', vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)

plt.tight_layout();

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(img_sharp, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Laplasijan sa dijagonalnim elemntima', fontsize=16)
ax[1].imshow(img_sharp2, cmap='gray', vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Laplasijan bez dijagonalnih elemnata', fontsize=16)

plt.tight_layout();

Desna slika subjektivno izgleda bolje, pa usvajamo taj rezultat.

In [ ]:
temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], img_sharp2))
img_out = color.hsv2rgb(temp)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgRGB, vmin=0, vmax=1); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(img_out, vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)

plt.tight_layout();

Problem kod ove metode je to što rezultujuća slika poprima "grainy" izgled, a jedini način da utičemo na to je podešavanjem jednog parametra. Tj glavna mana ove metode je manjak stepeni slobode.

Kao sledeću medtodu primenićemo unsharp mask, u opštem slučajuto predstavlja HP filtar, pa ćemo tako i implementirati ovu metodu. Imamo na raspolaganju 2 stepena slobode : 1) granica porpusnog opsega (indirektno se zadaje preko varijanse) 2) odnos snaga u propusnom/nepropusnom opsegu preko parametra aplha

Za radius usvajamo vrednost 10, zato što to ima efekat nalik na zero padding ako je sigma dovoljno malo, tj. neće značajno uticati na rezultat.

Počinjemo od aplha = 0.5, to predstavlja "pravi" HP filtar.

In [ ]:

def unsharp_mask(alpha = 0.8, sigma = 0.2 ,radius = 1):
    aplha = np.real(alpha)
    radius = np.real(radius)
    sigma = np.real(sigma)
    
    
    if(aplha > 1 or alpha < 0):
        print('Error : aplha must be between 0 and 1')
        return
    if(radius < 1):
        print('Error : radius must be positive')
        return
    if(sigma < 0):
        print('Error : sigma must be positive')
        return
    
    N = 2*radius + 1
    dirac = np.zeros((N, N))
    dirac[(N-1)//2, (N-1)//2] = 1
    
    w = aplha*dirac - (1-aplha)*skimage.filters.gaussian(dirac, sigma = sigma, truncate=N)
   
    return w

In [ ]:
alpha = 0.5
radius = 10

sigma = 0.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();




sigma = 1

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();




sigma = 10

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

Možemo primetiti sledeće: što je sigma veće to propuštamo sve veći i veći deo slike, razlog za to je LP filtar u frekv domenu postaje sve uži pa 1 - LP propušta sve veći deo spektra.

Takođe možemo primetit da je na ovaj način pojačavamo i šum, ali za raliku od laplasijana šum je manje izražen.

Posmatrajmo šta se dešava ako menjamo aplha za sigma = 0.5


In [ ]:
alpha = 0.1
radius = 10

sigma = 0.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

alpha = 0.499
radius = 10

sigma = 0.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

alpha = 0.5
radius = 10

sigma = 0.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

alpha = 0.501
radius = 10

sigma = 0.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

alpha = 0.9
radius = 10

sigma = 0.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

Kao što je i očekivano za male vrednosti alpha praktično dolazi samo do formiranja negativa NF dela slika, dok za velike vrednosti aplha praktično se ništa ne dešava.

Zanimljive su pojave za vrednosti aplha blizu 0.5, ali u kontekstu izoštravanja slike nam nisu od interesa.

In [ ]:
alpha = 0.5
radius = 10

sigma = 1.5

w = unsharp_mask(alpha, sigma, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(imgV + g, vmin=0, vmax=1,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)
plt.text(20, 60, f'alpha = {alpha}, sigma = {sigma}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

In [ ]:
temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], imgV + g))
img_out2 = color.hsv2rgb(temp)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgRGB, vmin=0, vmax=1); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(img_out2, vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)

plt.tight_layout();

Implementiraćemo još jednu metodu. Konstruisaćemo sharpening mask pomoću BP filtra kako bismo izbegli pojačajanje šuma, BP pravimo kao razliku 2 LP filtra.

In [ ]:
def band_pass_mask(sigma1 = 0.8, sigma2 = 0.2 ,radius = 1):
    sigma1 = np.real(sigma1)
    radius = np.real(radius)
    sigma2 = np.real(sigma2)
    
    
    if(sigma2 < 0):
        print('Error : sigma2 must be positive')
        return
    if(radius < 1):
        print('Error : radius must be positive')
        return
    if(sigma1 < 0):
        print('Error : sigma1 must be positive')
        return

    
    N = 2*radius + 1
    dirac = np.zeros((N, N))
    dirac[(N-1)//2, (N-1)//2] = 1
    
    w = skimage.filters.gaussian(dirac, sigma = sigma1, truncate=N) - skimage.filters.gaussian(dirac, sigma = sigma2, truncate=N)
   
    return w

In [ ]:

radius = 10

sigma1 = 0.1
sigma2 = 1

w = band_pass_mask(sigma1, sigma2, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'sigma1 = {sigma1}, sigma2 = {sigma2}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

radius = 10

sigma1 = 0.5
sigma2 = 1

w =band_pass_mask(sigma1, sigma2, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'sigma1 = {sigma1}, sigma2 = {sigma2}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

radius = 10

sigma1 = 0.9
sigma2 = 1

w = band_pass_mask(sigma1, sigma2, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(g,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('sharpening mask', fontsize=16)
plt.text(20, 60, f'sigma1 = {sigma1}, sigma2 = {sigma2}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

Možemo primetiti da što su sigma1 i sigma2 bliži to je filtriranje selektivnije.

In [ ]:
radius = 5

sigma1 = 0.5
sigma2 = 1.5

w = band_pass_mask(sigma1, sigma2, radius)
g = scipy.ndimage.correlate(imgV, w)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgV, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(imgV + g, vmin=0, vmax=1,cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)
plt.text(20, 60, f'sigma1 = {sigma1}, sigma2 = {sigma2}, radius = {round(radius)}' , fontsize = 14)



In [ ]:
temp = np.dstack((imgHSV[:,:, 0],imgHSV[:,:, 1], imgV + g))
img_out3 = color.hsv2rgb(temp)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(imgRGB, vmin=0, vmax=1); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika', fontsize=16)
ax[1].imshow(img_out3, vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('Izoštrena slika', fontsize=16)

plt.tight_layout();

sledi poređenje rezultata.

In [ ]:

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(img_out, vmin=0, vmax=1); ax[0].set_axis_off(); ax[0].set_title('Laplasijan', fontsize=16)
ax[1].imshow(img_out2, vmin=0, vmax=1); ax[1].set_axis_off(); ax[1].set_title('High pass', fontsize=16)
ax[2].imshow(img_out3, vmin=0, vmax=1); ax[2].set_axis_off(); ax[2].set_title('Band pass', fontsize=16)

plt.tight_layout();


Šum : BP->HP->Laplasijan


Jednostavnost: Laplasijan->HP->BP

## Treći zadatak

In [ ]:
corrupted = skimage.img_as_float(imread('sekvence/corrupted1.png'))

original = skimage.img_as_float(imread('sekvence/original1.png'))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(original, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('original1.jpg', fontsize=16)
ax[1].imshow(corrupted, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('corrupted1.jpg', fontsize=16)

plt.tight_layout();

Posmatajući degradiranu sliku možemo ustanoviti 2 stvari 1) Prisutan je izražen šum impulsne prirode 2) U pitanju je negativ originalne slike

Prvo analiziramo prirodu šuma posmatrajući uniforman deo slike, u ovom slučaju to je gornji levi ugao.

In [ ]:
[height, width] = shape(original)


fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(corrupted[10:height//8, 10:width//8], cmap = 'gray'); axes.set_axis_off(); axes.set_title('Gornji levi ugao corrupted slike')

Očigledno je u pitanju šum impulsne prirode.

In [ ]:
hist_s, bin_edges_s = np.histogram(corrupted[10:height//8, 10:width//8].flatten()*(256 - 1), bins=256, range=(0,(256 - 1)))
        
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=120);
plt.bar(bin_edges_s[0:-1], hist_s, color='b')

print(hist_s[0]/(sum(hist_s)))

Na osnovu histograma možemo proceniti da je verovatnoća impulsa P  = 0.026

In [ ]:

def adaptive_median(s, max_order = 3, start_order = 1):
    "3x3 -> order == 1, 5x5 --> order = 2, 7x7 -->order = 7x7 etc"
    max_order = int(max_order)
    start_order = int(start_order)
    if(max_order < 1):
        max_order = 1
    if(start_order < 1):
        start_order = 1
    if(start_order > max_order):
        start_order = max_order
        
    N_extra = max_order*2
    [Ny, Nx] = shape(s)
    Px = Nx + 2*N_extra; Py = Ny + 2*N_extra;
    
    img_pad = zeros([Py, Px], dtype = float)
    img_pad[N_extra : N_extra + Ny, N_extra :N_extra + Nx] = s
    
    
    
    for i in range(Nx):
        for j in range(Ny):
            increase_order = True
            order = start_order - 1
            check_xy = True
            
            x = i + N_extra
            y = j + N_extra
            
            while increase_order: 
                order = order + 1
                k = order

                max_px = np.max(img_pad[y - k : y + k  + 1, x - k : x + k + 1])
                min_px = np.min(img_pad[y - k : y + k  + 1, x - k : x + k + 1])
                med_px = np.median(img_pad[y - k : y + k  + 1, x - k : x + k + 1])
                
                
                if (med_px - min_px > 0) and (med_px - max_px < 0):
                    "Proveravamo da li je med zapravo impuls"
                    increase_order = False
                if(order >= max_order): 
                    check_xy = False
                    increase_order = False
                    img_pad[N_extra + j, N_extra + i] = med_px
                    
            if(check_xy):
                if (s[j, i] - min_px > 0) and (s[j, i] - max_px < 0):
                    img_pad[N_extra + j, N_extra + i] = s[j, i]
                else:
                    img_pad[N_extra + j, N_extra + i] = med_px 
                    "menja s[x, y] samo ako je impulsne prirode"
              
    return img_pad[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    



Adaptivni median filtar funkcioniše na sledeći način: Prvo odredi medijanu -> ako je medijana impulsne prirode (min ili max) proširi lokalnu okolinu. Postupak se ponavlja sve dok se ne dosegne max veličina okoline ili ne pronađe medijana koja nije impulsne prirode. 
Sledeći korak je samo filtriranje, ako piksel na poziciji i,j nije impulse prirode onda se ne menja, a ako jeste menja se sa medijanom.

Kao rezultat imamo mnogo bolje očuvanje ivica u poređenju sa klasičnim median filtrom, mana je to što se muči sa regionima koji treba da budu uniformni i imaju maksimalne ili minimalne vrednosti. 

In [ ]:
t = adaptive_median(corrupted, 2)


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(corrupted, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('corrupted', fontsize=16)
ax[1].imshow(t, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('filtrirano', fontsize=16)

plt.tight_layout();

Očigledno je u pitanju negativ slike, vršimo stransformaciju t = 1 - s

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(1-t, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('processed', fontsize=16)
ax[1].imshow(original, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('original', fontsize=16)

plt.tight_layout();

In [ ]:
plot_hist_img(original, 1 -  t, 256)

Možemo videti da je došlo do određene kompresije histograma, primenićemo linearnu transformaciju.

In [ ]:
#Piecewise linear histogram stretching

x_val = np.array([0, 49.9, 55,100, 170, 235.1, 255])/255
y_val = np.array([0, 49.9,20, 100, 235,235.1,255])/255

t2 = np.interp(double(1-t), x_val, y_val)




plot_hist_img(original,t2 , 256)

Malo ćemo da izoštrimo sliku.

In [ ]:
laplacian_mask = np.array([ [1,  1, 1],
                            [1, -8, 1],
                            [1,  1, 1] ])

out_lap = scipy.ndimage.correlate(t2, laplacian_mask)
out = t2 - 0.15*out_lap

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(out, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('reconstructed', fontsize=16)
ax[1].imshow(original, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('original', fontsize=16)

plt.tight_layout();

Slike izgledaju praktično identično, primenićemo isti postupak na druge dve slike pomoću funkcije reconstruction.

In [ ]:
#H = np.fft.fft2(t)/np.fft.fft2(original)

In [ ]:


def constrained_LS(img, gamma, H, k):
    [Ny, Nx] = shape(img)
    h_lap = [[1,  1, 1],
         [1, -8, 1],
         [1,  1, 1]]

    h_lap_p = zeros([Ny, Nx])
    h_lap_p[0:3,0:3] = h_lap

    P = np.fft.fft2(h_lap_p)
    
    F = np.conjugate(H)/(np.abs(H)**2 + gamma*np.abs(P)**2)*np.fft.fft2(img)
    
    return np.real(np.fft.ifft2(F*k))



def my_filter(img, H, gamma= 0.1, AM_order = 0,AM_start_order = 1,  sharpen = False):
    [NyH, NxH] = shape(H)
    [NyI, NxI] = shape(img)
    
    k = (NyH*NxH)/(NyI* NxI)
    
    if(NyI > NyH) or (NxI > NxH): 
        print('Image is larger than H, currently not supported')
        return
    
    Nx_extra = NxH - NxI
    Ny_extra = NyH - NyI
    
    Px = NxI + Nx_extra; Py = NyI + Ny_extra;
    
    img_pad = zeros([Py, Px])
    img_pad[ 0 : NyI, 0 : NxI] = img
    
    if (AM_order > 1):
        img_pad[ 0 : NyI, 0 : NxI] = adaptive_median(img_pad[ 0 : NyI, 0 : NxI], AM_order, AM_start_order)
        
    img_pad = constrained_LS(img_pad, gamma, H, k)
    
    out = img_pad[ 0 : NyI, 0 : NxI]
    
    
    
    if(np.median(out) < 0 or np.median(out) > 1):
        low_in = np.percentile(out.flatten(), .5)
        high_in = np.percentile(out.flatten(), 99.5)
        out = skimage.exposure.rescale_intensity(out, in_range=(low_in, high_in), out_range=(0,1))
    
    if(sharpen):
        laplacian_mask = np.array([ [1,  1, 1],
                            [1, -8, 1],
                            [1,  1, 1] ])

        out_lap = scipy.ndimage.correlate(out, laplacian_mask)
        out = out - out_lap
    

    return out 



In [ ]:
def reconstruction(img):
    
    x_val = np.array([0, 49.9, 55,100, 170, 235.1, 255])/255
    y_val = np.array([0, 49.9,20, 100, 235,235.1,255])/255
    t2 = adaptive_median(img, 1)
    t = np.interp(double(1-t2), x_val, y_val)
    
    laplacian_mask = np.array([ [1,  1, 1],
                            [1, -8, 1],
                            [1,  1, 1] ])

    out_lap = scipy.ndimage.correlate(t, laplacian_mask)
    t = t - 0.15*out_lap
    
    
    return t

In [ ]:
t = reconstruction(corrupted)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(corrupted, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('corrupted', fontsize=16)
ax[1].imshow(t, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('reconstructed', fontsize=16)

plt.tight_layout();


In [ ]:
corrupted = skimage.img_as_float(imread('sekvence/corrupted2.png'))
t2 = reconstruction(corrupted)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(corrupted, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('corrupted', fontsize=16)
ax[1].imshow(t2, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('reconstructed', fontsize=16)

plt.tight_layout();

In [ ]:
corrupted = skimage.img_as_float(imread('sekvence/corrupted3.png'))
t3 = reconstruction(corrupted)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(corrupted, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('corrupted', fontsize=16)
ax[1].imshow(t3, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('reconstructed', fontsize=16)

plt.tight_layout();

## Zadatak 4

In [ ]:
def bilateral_filter(x, radius, sigma_s, sigma_r):
    """
    Funkcija vrši filtriranje ulazne slike, uklanjanje šum uz maksimalno očuvanje ivica. 
    
    Parametar sigma_s određuje intezitet filtrianja šuma, što je sigma_s veće to je filtriranje agresivnije.
    Parametar sigma_r određuje stepen očuvanja ivica, što je sigma_r veće to se ivice MANJE čuvaju.
    
    Maska prostornog filtra je dimenzije (2*radius * 1)x(2*radius * 1), formira se kao:
    w(m, n) = exp(-(m^2 + n^2/sigma_s^2)exp(-((x(i, j) - x(i + m, j + n))^2/sigma_r^2)
    w(m, n) = w(m, n)/sum(w(m, n)) <- Normalizacija!
    
    Preporuka: radius treba birati rako da sigma_s ≈ radius
    
    Pre samog procesa filtriranja vrši se proširenje slike kopijama piksela sa ivice, tj. obrada se efektivno
    vrši nad slikom dimenzije (Ny + radius)x(Nx + radius)x1
     
    Ulazi
      x - realna matrica dimenzija NxMx1. Ulazna slika.
      radius - dimenzija filtra, pozitivan ceo broj
      sigma_s - pozitivan realni broj
      sigma_r - pozitivan realni broj
      
    Izlazi
      img_pad - realna matrica dimenzija NxMx1. Filtrirana slika.
      
      U slučaju greške napušta se funckija i ispisuje se poruka o grešci.
    
    Primer
        lena = skimage.img_as_float(imread('.../lena.png'))
        imgR = lena[:,:,0];
        
        imgR_filt = bilateral_filter(imgR, 5, 0.1, 0.1)

        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120); ax = axes.ravel();
        ax[0].imshow(imgR, vmin=0, vmax=1, cmap = 'gray'); ax[0].set_axis_off(); 
        ax[1].imshow(imgR_filt, vmin=0, vmax=1, cmap = 'gray'); ax[1].set_axis_off();

        
    Reference
        https://en.wikipedia.org/wiki/Bilateral_filter
    """
    
    sigma_s = np.real(sigma_s)
    sigma_r = np.real(sigma_r)
    x = np.real(x)
    radius = int(radius)
    
    if(sigma_s <= 0 ):
        print('Error: sigma_s must be positive')
        return
    if(sigma_r <= 0 or (not np.isreal(sigma_r))):
        print('Error: sigma_r must be positive')
        return
    if(np.min(x) <0 or np.max(x) > 1):
        print('Error: x must be from [0, 1]')
        return
    if(radius <= 0):
        print('Error: radius must be positive')
        return
    
    w = gaussian_mask(radius, sigma_s)
    z = np.zeros((2*radius + 1, 2*radius + 1)) + 1
    
    
    N_extra = radius
    [Ny, Nx] = shape(x)
    
    img_pad = np.pad(x, radius, mode='edge')
    img_pad[N_extra : N_extra + Ny, N_extra :N_extra + Nx] = x
    t = -1/(2*np.square(sigma_r))
    
    for i in range(Nx):
        for j in range(Ny):
            """ racuna se konvolucija maske w(m, n) i slike img_pad
                curr - je tekuci element sa pozicije j, i iz originalne slike
                T - je tezinski faktor koji zavisi od razlika u intezitetu T ~ exp(((x(i, j) - x(i + m, j + n))^2)
                norm - koeficijent za normalizaciju maske w
            """
            curr = np.array(img_pad[j + N_extra - radius : j + N_extra + radius  + 1, i + N_extra - radius : i + N_extra + radius + 1])
            T = w*exp(np.square(curr -  img_pad[j + N_extra, i + N_extra]*z)*t)
            norm = np.sum(T)
            if(norm):
                img_pad[j + N_extra, i + N_extra] = np.sum(T*curr)/norm
            else:
                img_pad[j + N_extra, i + N_extra] = np.sum(T*curr)
       
            
              
    return img_pad[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    
def gaussian_mask(radius, sigma_s):
    Nx = 2*radius + 1
    Ny = 2*radius + 1
    
    y = np.arange(0,Ny) - (Ny-1)/2
    x = np.arange(0,Nx) - (Nx-1)/2

    X, Y = meshgrid(x, y)
    
    D = np.square(X) + np.square(Y)
    
    return exp(-D/(2*np.square(sigma_s)))

In [ ]:
radius = [2, 4, 10, 30, 40, 100]

sigma_s = 0.1
sigma_r = 0.1

for i in radius:
    start = time.time()
    t = bilateral_filter(imgV, i, sigma_s, sigma_r)
    end = time.time()
    execution_time = (end - start)
    print('Vreme izvrsavanja: ' + str(round(execution_time,3))+ 's \n')
    execution_time_norm = execution_time/np.size(imgV)
    print('Vreme izvrsavanja: ' + str(round(execution_time_norm*1e6,3))+ 'us/pix \n')   

In [ ]:
radius = [2, 4, 10, 30, 40, 100]

sigma_s = 0.1
sigma_r = 0.1

for i in radius:
    start = time.time()
    t = skimage.restoration.denoise_bilateral(imgV, win_size = i, sigma_spatial = sigma_s, sigma_color = sigma_r, mode='edge')
    end = time.time()
    execution_time = (end - start)
    print('Vreme izvrsavanja: ' + str(round(execution_time,3))+ 's \n')
    execution_time_norm = execution_time/np.size(imgV)
    print('Vreme izvrsavanja: ' + str(round(execution_time_norm*1e6,3))+ 'us/pix \n')  

Očekivano je da je ugrađena funkcija brža zato što je implementirana u Cython-u, ali možemo primetiti da je asimprotski brža samo oko 3 - 4 puta. (Za male vrednosti radiusa je izraženo vreme koje se potroši na samo predprocesiranje, dok za veće je akcenat na složenosti algoritma za obradu) 

In [ ]:
sigma_s = 2
sigma_r = 1
radius = 2
t = bilateral_filter(imgV, radius, sigma_s, sigma_r)

t1= skimage.restoration.denoise_bilateral(imgV, win_size = radius, sigma_spatial = sigma_s, sigma_color = sigma_r, mode='edge')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(t, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('bilateral_filter', fontsize=16)
ax[1].imshow(t1,  cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('skimage_bilateral_filter', fontsize=16)
plt.text(20, 60, f'sigma_s = {sigma_s}, sigma_r = {sigma_r}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

sigma_s = 2
sigma_r = 0.1
radius = 2
t = bilateral_filter(imgV, radius, sigma_s, sigma_r)

t1= skimage.restoration.denoise_bilateral(imgV, win_size = radius, sigma_spatial = sigma_s, sigma_color = sigma_r, mode='edge')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(t, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('bilateral_filter', fontsize=16)
ax[1].imshow(t1,  cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('skimage_bilateral_filter', fontsize=16)
plt.text(20, 60, f'sigma_s = {sigma_s}, sigma_r = {sigma_r}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

sigma_s = 2
sigma_r = 0.01
radius = 2
t = bilateral_filter(imgV, radius, sigma_s, sigma_r)

t1= skimage.restoration.denoise_bilateral(imgV, win_size = radius, sigma_spatial = sigma_s, sigma_color = sigma_r, mode='edge')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,9), dpi=120)
ax = axes.ravel()

ax[0].imshow(t, cmap = 'gray'); ax[0].set_axis_off(); ax[0].set_title('bilateral_filter', fontsize=16)
ax[1].imshow(t1,  cmap = 'gray'); ax[1].set_axis_off(); ax[1].set_title('skimage_bilateral_filter', fontsize=16)
plt.text(20, 60, f'sigma_s = {sigma_s}, sigma_r = {sigma_r}, radius = {radius}' , fontsize = 14)

plt.tight_layout();

Implementirana funkcija se ponaša očekivano, što je sigma_r manje to je veći akcenat na očuvanju ivica. Ugrađena funkcija se ponaša kao da ima clipping za parametar sigma_r, što je jako neobično zato što to nije naglašeno u opsiu funkcije.